In [41]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import tqdm
import gpxpy
import gpxpy.gpx
import os
from datetime import datetime, timezone


In [44]:
SG_data_path = "D:/repos/Grab_ETA_Prediction/data/grab-posis-city=Singapore/city=Singapore"
dataset = pq.ParquetDataset(SG_data_path)
table = dataset.read()


In [47]:
pd_table

,trj_id,rawlat,rawlng,pingtimestamp,speed
0,70014,1.342326,103.888969,1554943236,18.910000
1,73573,1.321781,103.856366,1555582623,17.719076
2,75567,1.327088,103.861273,1555141026,14.021548
3,1410,1.262482,103.823794,1555731693,13.026521
4,4354,1.283799,103.807210,1555584497,14.812943
...,...,...,...,...,...
30329680,72196,1.326351,103.890147,1554771148,5.880000
30329681,58589,1.322549,103.970248,1555842124,25.285847
30329682,80684,1.300553,103.907405,1554720657,20.533981
30329683,498,1.347160,103.966904,1554990704,25.770779


In [46]:
pd_table = table.to_pandas()
gpx_traces = np.array([])
NUM_TRIPS_TO_ANALYZE = 1
# TRJ_IDX, LAT_IDX, LNG_IDX = 0, 4, 5
pd_table = pd_table[["trj_id", "rawlat", "rawlng", "pingtimestamp", "speed"]]
TRJ_IDX, LAT_IDX, LNG_IDX, PINGTIMESTAMP_IDX, SPEED_IDX = 0, 1, 2, 3, 4

In [48]:
all_trj_ids = pd_table.trj_id.unique()

In [49]:
def extract_xml(trj_id, trj_gps_list):
    gpx = gpxpy.gpx.GPX()

    gpx_track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track)

    # Create first segment in our GPX track:
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)

    for gps in trj_gps_lst:
        lat, lng, speed, pingtime = gps[[LAT_IDX, LNG_IDX, SPEED_IDX, PINGTIMESTAMP_IDX]]
        pingtime = datetime.datetime.utcfromtimestamp(float(pingtime))
        gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(lat, lng, speed=speed, time=pingtime, elevation=1234))
#     print('Created GPX:', gpx.to_xml())
    
    folder_to_save_path = "D:/repos/third-party/map-matching/matching-web/src/test/resources/grab_sg"
    file_name = trj_id + ".gpx"
    f = open(os.path.join(folder_to_save_path, file_name), "w")
    f.write(gpx.to_xml(version="1.0"))
    f.close()


In [ ]:
np_table = pd.DataFrame.to_numpy(pd_table)


In [90]:
for gps_samp in np_table:
    lat, lng = gps_samp[[LAT_IDX, LNG_IDX]]
    print(get_shortest_edge(G, lat, lng))

475378464-0
546689528-0
316209674-0
257183870-0
422962325-4
455246818-22


KeyboardInterrupt: 

In [ ]:
grouped = pd_table.sort_values(["pingtimestamp"]).groupby("trj_id")


In [149]:
# trj_gps_lst = pd_table[pd_table["trj_id"] == trj_id].sort_values(["pingtimestamp"])
# trj_gps_lst = pd.DataFrame.to_numpy(trj_gps_lst)

for trj_id, trj_gps_lst in tqdm.tqdm(grouped):
#     print(trj_id)
    trj_gps_lst = pd.DataFrame.to_numpy(trj_gps_lst)
    extract_xml(trj_id, trj_gps_lst)

100%|████████████████████████████████████████████████████████████████████████████| 28000/28000 [20:26<00:00, 22.83it/s]


In [85]:
def get_distance(lat, lng, lat1, lng1, lat2, lng2):
    return abs((lng2-lng1) * lat - (lat2 - lat1) * lng + lat2*lng1 - lat1*lng2) \
/ np.sqrt((lat2-lat1)**2 +(lng2-lng1)**2)
    
def get_shortest_edge(G, lat, lng):
    distance_arr = [get_distance(lat, lng, 
                                 G.node[nodeA]["data"].lat, G.node[nodeA]["data"].lon,
                                 G.node[nodeB]["data"].lat, G.node[nodeB]["data"].lon)
                    for nodeA, nodeB in G.edges()]
    nodeA, nodeB = G.edges()[np.argmin(distance_arr)]
    edge = G.get_edge_data(nodeA, nodeB)["data"]
    return edge.id


In [86]:
get_shortest_edge(G, 0, 0)

'758639039-0'

In [81]:
nodeA, nodeB = G.edges()[np.argmin(distance_arr)]
edge = G.get_edge_data(nodeA, nodeB)["data"]
edge.id

'758639039-0'

In [21]:
osm_file_xml_path = "D:/repos/third-party/map-matching/map-data/Singapore.osm/Singapore.osm"
G = read_osm(osm_file_xml_path)

In [74]:
print(len(G.nodes()))
print(len(G.edges()))
print(G.edges()[0])
G.get_edge_data('4748500344', '6427627507')["data"].nds
G.node["4748500344"]["data"].lat

544198
597272
('4748500344', '6427627507')


1.3060184

In [20]:
import xml.sax
import copy
import networkx

def download_osm(left,bottom,right,top):
    """ Return a filehandle to the downloaded data."""
    from urllib.request import urlopen
    print( "http://api.openstreetmap.org/api/0.6/map?bbox=%f,%f,%f,%f"%(left,bottom,right,top))
    fp = urlopen( "http://api.openstreetmap.org/api/0.6/map?bbox=%f,%f,%f,%f"%(left,bottom,right,top) )
    return fp

def read_osm(filename_or_stream, only_roads=True):
    """Read graph in OSM format from file specified by name or by stream object.
    Parameters
    ----------
    filename_or_stream : filename or stream object
    Returns
    -------
    G : Graph
    Examples
    --------
    >>> G=nx.read_osm(nx.download_osm(-122.33,47.60,-122.31,47.61))
    >>> plot([G.node[n]['data'].lat for n in G], [G.node[n]['data'].lon for n in G], ',')
    """
    osm = OSM(filename_or_stream)
    G = networkx.Graph()

    for w in osm.ways.values():
        if only_roads and 'highway' not in w.tags:
            continue
        G.add_path(w.nds, id=w.id, data=w)
    for n_id in G.nodes_iter():
        n = osm.nodes[n_id]
        G.node[n_id] = dict(data=n)
    return G
        
    
class Node:
    def __init__(self, id, lon, lat):
        self.id = id
        self.lon = lon
        self.lat = lat
        self.tags = {}
        
class Way:
    def __init__(self, id, osm):
        self.osm = osm
        self.id = id
        self.nds = []
        self.tags = {}
        
    def split(self, dividers):
        # slice the node-array using this nifty recursive function
        def slice_array(ar, dividers):
            for i in range(1,len(ar)-1):
                if dividers[ar[i]]>1:
                    #print "slice at %s"%ar[i]
                    left = ar[:i+1]
                    right = ar[i:]
                    
                    rightsliced = slice_array(right, dividers)
                    
                    return [left]+rightsliced
            return [ar]
            
        slices = slice_array(self.nds, dividers)
        
        # create a way object for each node-array slice
        ret = []
        i=0
        for slice in slices:
            littleway = copy.copy( self )
            littleway.id += "-%d"%i
            littleway.nds = slice
            ret.append( littleway )
            i += 1
            
        return ret
        
        

class OSM:
    def __init__(self, filename_or_stream):
        """ File can be either a filename or stream/file object."""
        nodes = {}
        ways = {}
        
        superself = self
        
        class OSMHandler(xml.sax.ContentHandler):
            @classmethod
            def setDocumentLocator(self,loc):
                pass
            
            @classmethod
            def startDocument(self):
                pass
                
            @classmethod
            def endDocument(self):
                pass
                
            @classmethod
            def startElement(self, name, attrs):
                if name=='node':
                    self.currElem = Node(attrs['id'], float(attrs['lon']), float(attrs['lat']))
                elif name=='way':
                    self.currElem = Way(attrs['id'], superself)
                elif name=='tag':
                    self.currElem.tags[attrs['k']] = attrs['v']
                elif name=='nd':
                    self.currElem.nds.append( attrs['ref'] )
                
            @classmethod
            def endElement(self,name):
                if name=='node':
                    nodes[self.currElem.id] = self.currElem
                elif name=='way':
                    ways[self.currElem.id] = self.currElem
                
            @classmethod
            def characters(self, chars):
                pass

        xml.sax.parse(filename_or_stream, OSMHandler)
        
        self.nodes = nodes
        self.ways = ways
            
        #count times each node is used
        node_histogram = dict.fromkeys( self.nodes.keys(), 0 )
        for way in list(self.ways.values()):
            if len(way.nds) < 2:       #if a way has only one node, delete it out of the osm collection
                del self.ways[way.id]
            else:
                for node in way.nds:
                    node_histogram[node] += 1
        
        #use that histogram to split all ways, replacing the member set of ways
        new_ways = {}
        for id, way in self.ways.items():
            split_ways = way.split(node_histogram)
            for split_way in split_ways:
                new_ways[split_way.id] = split_way
        self.ways = new_ways